In [2]:
import cv2
import numpy as np
import mediapipe as mp
import os
import pandas as pd

In [3]:
MP_Pose = mp.solutions.pose
Pose = MP_Pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6 )

In [4]:
Main_Angles = {
    
    "L_Neck"     : [0,11,12]  , 
    "R_Neck"     : [0,12,11]  , 
    "L_Shoulder" : [13,11,23] , 
    "R_Shoulder" : [14,12,24] , 
    "L_Elbow"    : [11,13,15] , 
    "R_Elbow"    : [12,14,16] ,
    "L_Wrist"    : [13,15,19] , 
    "R_Wrist"    : [14,16,20] ,
    "L_Hip"      : [11,23,25] , 
    "R_Hip"      : [12,24,26] ,
    "L_Knee"     : [23,25,27] ,
    "R_Knee"     : [24,26,28] ,
    "L_Ankle"    : [25,27,31] ,
    "R_Ankle"    : [26,28,32] , 
}


In [6]:
def Calculate_Angle(Point_1, Point_2, Point_3):
    Point_1 = np.array(Point_1) 
    Point_2 = np.array(Point_2)
    Point_3 = np.array(Point_3) 
    
    radians = np.arctan2(Point_3[1]-Point_2[1], Point_3[0]-Point_2[0]) - np.arctan2(Point_1[1]-Point_2[1], Point_1[0]-Point_2[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

### Create file with Num of poses for each exercise

In [5]:
folder_path = "Data//Exercises"

# Dictionary to store exercise names and pose counts
exercise_dict = {}

for filename in os.listdir(folder_path):
        
        parts = filename.split()
        exercise_name = " ".join(parts[:-1])
        pose_number = parts[-1]
        exercise_key = f"{exercise_name}"

        if exercise_key in exercise_dict:
            exercise_dict[exercise_key] += 1
        else:
            exercise_dict[exercise_key] = 1

df = pd.DataFrame(list(exercise_dict.items()), columns=["Exercise Name", "Poses Count"])

excel_filename = "Data//Pose Counts.xlsx"
df.to_excel(excel_filename, index=False)

print(f"Excel file '{excel_filename}' created successfully.")


Excel file 'Data//Pose Counts.xlsx' created successfully.


### Calculate Main Angles from Excercises images

In [6]:
def Calculate_Main_Angles(image_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = Pose.process(image)

    Angles = []

    if results.pose_landmarks:
        L_Marks = results.pose_landmarks.landmark

        for Key, Val in Main_Angles.items():
            Angle = int(Calculate_Angle([L_Marks[Val[0]].x, L_Marks[Val[0]].y],
                                        [L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                        [L_Marks[Val[2]].x, L_Marks[Val[2]].y]))
            Angles.append(Angle)

    return Angles


### Create file with exercises' extracted angles

In [7]:
def Extract_Exercises_Features(folder_path):

    df = pd.read_excel("Data//Pose Counts.xlsx")

    exercise_angles_list = []

    for index, row in df.iterrows():
        exercise_name = row['Exercise Name']
        pose_count = row['Poses Count']

        # Calculate main angles for each pose
        pose_angles = []

        for pose in range(1, pose_count + 1):
            image_path = os.path.join(folder_path, f"{exercise_name} {pose}.png")
            landmarks = Calculate_Main_Angles(image_path)

            # Check if landmarks are detected
            if landmarks:
                pose_angles.append(landmarks)
            else:
                print(f"No landmarks detected for {exercise_name} pose {pose}. Skipping.")
                

        # Append exercise information to the list
        exercise_angles_list.append([exercise_name, pose_angles])

    # Create a DataFrame from the list
    df_result = pd.DataFrame(exercise_angles_list, columns=["Exercise Name", "Pose Angles"])

    # Save the DataFrame to an Excel file
    df_result.to_excel("Data//Exercises Features.xlsx", index=False)

    print("Excel file created successfully.")

Extract_Exercises_Features("Data//Exercises")


Excel file created successfully.


# Testing Functions

### Draw Angles on Exercises images 

In [11]:
def Draw_Angles( image , L_Marks ) :
    
    height, width, _ = image.shape
    for Key, Val in Main_Angles.items():
        
        Angle = int(Calculate_Angle([L_Marks[Val[0]].x, L_Marks[Val[0]].y],
                                    [L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                    [L_Marks[Val[2]].x, L_Marks[Val[2]].y]))
        txt = tuple(np.multiply([L_Marks[Val[1]].x, L_Marks[Val[1]].y],[width,height]).astype(int))

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        font_thickness = 2
        text_color = (209, 196, 233) 

        text_size, _ = cv2.getTextSize(str(Angle), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        cv2.rectangle(image, txt, (txt[0] + text_size[0], txt[1] - text_size[1]),(0, 0, 102), cv2.FILLED)

        cv2.putText(image, str(Angle), txt, font, font_scale, text_color, font_thickness, cv2.LINE_AA)
    return image


def Draw_Main_Angles(image_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = Pose.process(image)

    if results.pose_landmarks:
        print("Pose landmarks detected.")
        L_Marks = results.pose_landmarks.landmark
        image = Draw_Angles(image, L_Marks)
    else:
        print("No pose landmarks detected.")
        
    cv2.imshow('phlex', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    key = cv2.waitKey(0)
    if key == 27:
        cv2.destroyAllWindows()

Draw_Main_Angles('Data//Exercises//Bridge 2.png')

Pose landmarks detected.


: 